In [1]:
from __future__ import print_function

In [2]:
from ops import *

Using TensorFlow backend.


In [3]:
import os

In [4]:
from keras.layers.convolutional import Convolution2D, Deconvolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation, Lambda
from keras.layers import Input, merge
from keras.regularizers import l2
from keras.models import Model
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input
import keras.backend as K
import numpy as np
import h5py


In [5]:
color_image_path = r"C:\Users\Promise\Desktop\AI Projects\Sketchback\dataset\training\resized\a\f-005-01.png"
sketch_image_path = r"C:\Users\Promise\Desktop\AI Projects\Sketchback\dataset\training\resized\b\f-005-01.png"

In [6]:
from PIL import Image
import matplotlib.pyplot as plt

In [7]:
import matplotlib.image as mpimg

In [8]:
img = mpimg.imread(color_image_path)

In [9]:
img.shape

(256, 256, 3)

In [10]:
sketch = mpimg.imread(sketch_image_path)
print (sketch.shape)
plt.imshow(sketch[:,:,0])

(256, 256, 3)


In [11]:
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input
from copy import deepcopy

In [12]:
base_model = vgg16.VGG16(weights='imagenet', include_top=False)
vgg = Model(input=base_model.input, output=base_model.get_layer('block2_conv2').output)

C:\Users\Promise\Miniconda2\envs\ztdl\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  


In [13]:
def preprocess_vgg(x, data_format=None):
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}
    x = 255. * x
    if data_format == 'channels_first':
        # 'RGB'->'BGR'
        x = x[:, ::-1, :, :]
        # Zero-center by mean pixel
        x[:, 0, :, :] = x[:, 0, :, :] - 103.939
        x[:, 1, :, :] = x[:, 1, :, :] - 116.779
        x[:, 2, :, :] = x[:, 2, :, :] - 123.68
    else:
        # 'RGB'->'BGR'
        x = x[:, :, :, ::-1]
        # Zero-center by mean pixel
        x[:, :, :, 0] = x[:, :, :, 0] - 103.939
        x[:, :, :, 1] = x[:, :, :, 1] - 116.779
        x[:, :, :, 2] = x[:, :, :, 2] - 123.68
    return x

In [14]:
def get_features(Y):
    Z = deepcopy(Y)
    Z = preprocess_vgg(Z)
    features = vgg.predict(Z, batch_size = 5, verbose = 0)
    return features

In [15]:
feature = get_features(img.reshape(-1,256,256,3))

In [16]:
# feature = feature.reshape(256,256,128)

In [17]:
m = 256
n = 256
image_path = r"C:\Users\Promise\Desktop\AI Projects\Sketchback\dataset\training\resized\a"
sketch_path = r"C:\Users\Promise\Desktop\AI Projects\Sketchback\dataset\training\resized\b"
batch_size = 10

In [18]:
def load_file_names(path):
    return os.listdir(path)

In [19]:
def get_data(X = True, Y = True, W = True):
    
    x_path = sketch_path
    y_path = image_path
    file_names = load_file_names(x_path)

    X_train = np.zeros((len(file_names), m, n), dtype='float32')
    Y_train = np.zeros((len(file_names), m, n, 3), dtype='float32')
    F_train = None

    
    
        
    if X:
        # Load Sketches
        for i in range(len(file_names)):
            file = os.path.join(x_path, file_names[i])
            img = mpimg.imread(file)
            img = img.astype('float32')
            try:
                X_train[i] = img[:,:,0] / 255.
            except:
                X_train[i] = img[:,:] / 255
            
    if Y:
        # Load Ground-truth Images
        for i in range(len(file_names)):
            file = os.path.join(y_path, file_names[i])
            img = mpimg.imread(file)
            img = img.astype('float32')
            Y_train[i] = img / 255.
    
    if W:
        F_train = get_features(Y_train)
    
    X_train = np.reshape(X_train, (len(file_names), m, n, 1))
    return X_train, Y_train, F_train

In [26]:
from keras.optimizers import Adam
from keras.utils import generic_utils
from ops import *
import keras.backend as K
from keras.utils.vis_utils import plot_model
import sys
# Utils
sys.path.append("utils/")
from simple_utils import plot_batch_train, plot_batch_eval

import os
import time

In [27]:
def feature_loss(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))


def pixel_loss(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))


def variation_loss(y_true, y_pred):
    # Assume img size is 64*64
    if K.image_dim_ordering() == 'tf':
        a = K.square(y_pred[:, :64-1, :64-1, :] - y_pred[:, 1:, :64-1, :])
        b = K.square(y_pred[:, :64-1, :64-1, :] - y_pred[:, :64-1, 1:, :])
    else:
        a = K.square(y_pred[:, :, 64 - 1, :64 - 1] - y_pred[:, :, 1:, :64 - 1])
        b = K.square(y_pred[:, :, 64 - 1, :64 - 1] - y_pred[:, :, :64 - 1, 1:])
    return K.sum(K.sqrt(a+b))


In [31]:
def train(batch_size, n_batch_per_epoch, nb_epoch, sketch, color, weights, save_weight=1, img_dim=[256,256,1]):
    opt = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model, model_name = edge2color(img_dim, batch_size=batch_size)

    model.compile(loss=[pixel_loss, feature_loss], loss_weights=[1, 1], optimizer=opt)
    model.summary()
#     plot_model(model, to_file='figures/edge2color.png', show_shapes=True, show_layer_names=True)

    global_counter = 1
    for epoch in range(nb_epoch):
        batch_counter = 1
        start = time.time()
        batch_idxs = sketch.shape[0] // batch_size
        if n_batch_per_epoch >= batch_idxs or n_batch_per_epoch == 0:
            n_batch_per_epoch = batch_idxs
        progbar = generic_utils.Progbar(n_batch_per_epoch * batch_size)

        sk_val = sketch[0:16]
        co_val = color[0:16]
        sketch = sketch[16:]
        color = color[16:]
        weights = weights[16:]

        for idx in range(batch_idxs):
            batch_sk = sketch[idx * batch_size: (idx + 1) * batch_size]
            batch_co = color[idx * batch_size: (idx + 1) * batch_size]
            batch_weights = weights[idx * batch_size: (idx + 1) * batch_size]
            train_loss = model.train_on_batch([batch_sk], [batch_co, batch_weights])
            batch_counter += 1
            progbar.add(batch_size, values=[('pixel_loss', train_loss[1]), ('feature_loss', train_loss[2])])
            # if batch_counter >= n_batch_per_epoch:
            if global_counter % 50 == 1:
               
                global_counter = global_counter + 1

            if batch_counter >= n_batch_per_epoch:
                break
        print ("")
        print ('Epoch %s/%s, Time: %s' % (epoch + 1, nb_epoch, time.time() - start))

        if save_weight:
            # save weights every epoch
            weights_path = '%s/%s_weights_epoch_%s.h5' % (model_name, tag, epoch)
            if not os.path.exists('%s' % model_name):
                os.mkdir('%s' % model_name)
            model.save_weights(weights_path, overwrite=True)


In [32]:
batch_size, n_batch_per_epoch, nb_epoch = 30, 10, 20

In [24]:
sketches, colors, features = get_data()

In [30]:
train(batch_size, n_batch_per_epoch, nb_epoch, sketches, colors, features)

C:\Users\Promise\Desktop\AI Projects\Convolutional_Sketch_Inversion\src\ops.py:77: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (9, 9), name="block1_conv2D", padding="same", strides=(1, 1))`
  x = Convolution2D(nb_filter, k_size, k_size, name=name, border_mode="same", subsample=subsample)(x)
C:\Users\Promise\Desktop\AI Projects\Convolutional_Sketch_Inversion\src\ops.py:77: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), name="block2_conv2D", padding="same", strides=(2, 2))`
  x = Convolution2D(nb_filter, k_size, k_size, name=name, border_mode="same", subsample=subsample)(x)
C:\Users\Promise\Desktop\AI Projects\Convolutional_Sketch_Inversion\src\ops.py:77: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), name="block3_conv2D", padding="same", strides=(2, 2))`
  x = Convolution2D(nb_filter, k_size, k_size, name=name, border_mode="same", subsample=subsample)(x)
C:\Users\Promise\Desktop\AI Projects\Convo

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 256, 256, 1)       0         
_________________________________________________________________
block1_conv2D (Conv2D)       (None, 256, 256, 32)      2624      
_________________________________________________________________
batch_normalization_7 (Batch (None, 256, 256, 32)      1024      
_________________________________________________________________
activation_7 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
block2_conv2D (Conv2D)       (None, 128, 128, 64)      18496     
_________________________________________________________________
batch_normalization_8 (Batch (None, 128, 128, 64)      512       
_________________________________________________________________
activation_8 (Activation)    (None, 128, 128, 64)      0         
__________

C:\Users\Promise\Desktop\AI Projects\Convolutional_Sketch_Inversion\src\ops.py:152: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  vgg_first2 = Model(input=vgg_16.input, output=vgg_16.get_layer('block2_conv2').output)
C:\Users\Promise\Desktop\AI Projects\Convolutional_Sketch_Inversion\src\ops.py:158: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="edge2color", outputs=[<tf.Tenso..., inputs=Tensor("in...)`
  model = Model(input=x_input, output=[h11, feat], name='edge2color')


AttributeError: module 'pydot' has no attribute 'find_graphviz'